In [1]:
import numpy as np
import wave
import librosa
import scipy.io.wavfile
from scipy.fftpack import dct
from pydub import AudioSegment
from pydub.silence import split_on_silence
import matplotlib.pyplot as plt
import madmom
import scipy.signal
import os
from scipy.fftpack import fft, ifft,fftshift,fftfreq
from scipy.signal import butter, lfilter, freqz



intrusion_sounds=os.listdir("Dataset/1_intrusion")




In [2]:
#get the audio
#convert it into 5 sec i.e. if short:pad   else trim
audio_file_name="1.wav"
signal, sample_rate = librosa.load(audio_file_name, res_type='kaiser_fast')

def resize_audio(signal,sample_rate,time=5):
    
    if((len(signal)/sample_rate) < time):
        no_pads=time *sample_rate -len(signal)
        pads=np.zeros(no_pads)
        signal=np.append(signal,pads)
        
    else:
        signal=signal[0:int(time*sample_rate)]
    
resize_audio(signal,sample_rate)   

In [3]:
#Signal pre-emphasis
#signal, sample_rate = librosa.load(audio_file_name, res_type='kaiser_fast')

signal, sample_rate = madmom.audio.signal.load_wave_file(audio_file_name)
def emphasize_signal(signal,pre_emphasis = 0.97):
    return np.append(signal[0], signal[1:] - pre_emphasis * signal[:-1])

signal=emphasize_signal(signal)

Steps:-

0)Resample,attenuate,normalize etc the signal to sample_rate=48000Hz

1)Divide the audio into frames with han windows and 

2)then perform noise reduction ,

3)then combine the signal again and

4)divide it into sound chunks for further classification

In [4]:
#signal=madmom.audio.signal.Signal(audio_file_name,48000)
#Dont't run hangs the window   

#signal = scipy.signal.resample(signal,8000)
#print(len(signal)/sample_rate)


In [5]:

#To split the signal into frames of 2048 samples length which overlap by 0 samples(for now only to do noise reduction)
fs = madmom.audio.signal.FramedSignal(signal, frame_size=2048, hop_size=1)


In [6]:
audio_file_name="1.wav"
signal, sample_rate = librosa.load(audio_file_name, res_type='kaiser_fast')
emphasized_signal=emphasize_signal(signal)

def create_frames(emphasized_signal,size=2048,sample_rate=48000):
    #Framing the audio signal
    frame_size=size/sample_rate #24ms                        128=sample_rate*time    thus time=128/sample_rate
    frame_overlap=frame_size/2 #15ms overlap( ~50%)
    frame_length=int(round(frame_size*sample_rate))
    stride_length=int(round((frame_overlap)*sample_rate))
    # Pad the emphasized_signal with zeros in end corresponding to frame size
    signal_length=len(emphasized_signal)
    print("signal_length:"+str(signal_length)+"  frame_length:"+str(frame_length)+"  stride_length:"+str(stride_length))

    no_strides=np.ceil((signal_length/stride_length))
    print("no_strides:"+str(no_strides))
    no_pads=int(abs(signal_length-no_strides*stride_length))
    print("no_pads:"+str(no_pads))
    z=np.zeros(no_pads)
    print(np.shape(z))
    emphasized_signal=np.append(emphasized_signal,z)
    no_frames=int(len(emphasized_signal))/(stride_length)
    print("no_frames:"+str(no_frames))
    print(np.shape(emphasized_signal))

    frames=[]
    counter=0
    for i in range(0,len(signal)):
        #frames[counter]=signal[i:(i+frame_length)]
        frames.insert(counter, signal[i:(i+frame_length)])
        i+=stride_length
        counter+=1
    
    return frames

frames=create_frames(emphasized_signal,2048,48000)

signal_length:4105583  frame_length:2048  stride_length:1024
no_strides:4010.0
no_pads:657
(657,)
no_frames:4010.0
(4106240,)


In [7]:
def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(frame, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, frame)
    return y

def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def butter_highpass_filter(frame, cutoff, fs, order=5):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = lfilter(b, a, frame)
    return y



'''
def apply_low_pass_filter(frame,sample_rate=48000,cutoff=3300):
    y=butter_lowpass_filter(frame, cutoff, sample_rate, order=5)
    return y

def apply_high_pass_filter(frame,sample_rate=48000,cutoff=3300):
    y=butter_highpass_filter(frame, cutoff, sample_rate, order=5)
    return y
'''
    

'\ndef apply_low_pass_filter(frame,sample_rate=48000,cutoff=3300):\n    y=butter_lowpass_filter(frame, cutoff, sample_rate, order=5)\n    return y\n\ndef apply_high_pass_filter(frame,sample_rate=48000,cutoff=3300):\n    y=butter_highpass_filter(frame, cutoff, sample_rate, order=5)\n    return y\n'

In [ ]:
def remove_noise(frames,sample_rate=48000,low_cutoff=3000,high_cutoff=800):
    denoised_frames=[]
    for i in range(0,len(frames)):
        #b,a=butter(5, normal_cutoff, btype='low', analog=False)
        #fft=scipy.fftpack.fft(frames[i])
        filtered_signal=butter_lowpass_filter(frames[i],low_cutoff,sample_rate,5)   #select frequencies below 3000Hz
        filtered_signal=butter_highpass_filter(filtered_signal,high_cutoff,sample_rate,5) #select frequencies above 800Hz
        frames[i]=filtered_signal
        
remove_noise(frames)        